In [1]:
import tqdm
import requests

from pathlib import Path
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch("http://localhost:9200")

print(es.ping())
print(es.info())

True
{'name': 'fe4eb5f4ec55', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'sYghMxR5T_uCxUjyWhJEdg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [4]:
es_url = "http://localhost:9200"

In [5]:
requests.get(es_url).json()

{'name': 'fe4eb5f4ec55',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'sYghMxR5T_uCxUjyWhJEdg',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [22]:
response = requests.put(
    url=f"{es_url}/test-index-01",
    json={
        "mappings": {
            "properties": {
                "act_title": {"type": "text"},
                "content": {"type": "text"}
            }
        }
    }
)

response.json()

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-index-01'}

In [47]:
response = requests.put(
    url=f"{es_url}/articles-index",
    json={
        "settings": {
            "analysis": {
                "analyzer": {
                    "polish-law-analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "synonym-filter",
                            "morfologik_stem",
                            "lowercase"
                        ]
                    }
                },
                "filter": {
                    "synonym-filter": {
                        "type": "synonym",
                        "expand": False,
                        "synonyms": [
                            "kpk => kodeks postępowania karnego",
                            "kpc => kodeks postępowania cywilnego",
                            "kk => kodeks karny",
                            "kc => kodeks cywilny"
                        ]
                    }
                }
            }
        }
    }
)

response.json()

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'articles-index'}

In [49]:
response = requests.put(
    url=f"{es_url}/articles-index/_mapping",
    json={
        "properties": {
            "act_title": {
                "type": "text",
                "analyzer": "polish-law-analyzer"
            },
            "content": {
                "type": "text",
                "analyzer": "polish-law-analyzer"
            }
        }
    }
)

response.json()

{'acknowledged': True}

In [50]:
response = requests.get(
    url=f"{es_url}/articles-index/_analyze",
    json={
        "text": "jestem kpc",
        "analyzer": "polish-law-analyzer"
    }
)

response.json()

{'tokens': [{'token': 'być',
   'start_offset': 0,
   'end_offset': 6,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'kodeks',
   'start_offset': 7,
   'end_offset': 10,
   'type': 'SYNONYM',
   'position': 1},
  {'token': 'postępowanie',
   'start_offset': 7,
   'end_offset': 10,
   'type': 'SYNONYM',
   'position': 2},
  {'token': 'postępować',
   'start_offset': 7,
   'end_offset': 10,
   'type': 'SYNONYM',
   'position': 2},
  {'token': 'cywilny',
   'start_offset': 7,
   'end_offset': 10,
   'type': 'SYNONYM',
   'position': 3}]}

In [51]:
# TODO: dodać wszystkie ustawy do indeksu